# Warfighter PPO

Reference: https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Python-LLAPI.md

In [ ]:
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.environment import ActionTuple
import numpy as np
import random

In [ ]:
# This is a non-blocking call that only loads the environment.
env = UnityEnvironment(seed=1, side_channels=[])

# Start interacting with the environment.
env.reset()
behavior_names = env.behavior_specs
for name in behavior_names:
    print(name)

for i in range(0, 10):
    action_tuple = ActionTuple()
    action_tuple.add_continuous(random.uniform(0, 1))
    action_tuple.add_continuous(random.uniform(0, 1))
    env.set_actions("Warfighter?team=0", action_tuple)
    env.step()

# Close connection
env.close()